In [1]:
from tqdm import tqdm

face_identity_labelers = Labeler.objects.filter(
    name__startswith='face-identity'
).exclude(
    name__startswith='face-identity-old'
).exclude(name__startswith='face-identity-rekognition')
rekognition_labelers = [
    Labeler.objects.get(name='face-identity-rekognition'),
    Labeler.objects.get(name='face-identity-rekognition:augmented-l2-dist=0.7')
]
    
sampler_3s = FrameSampler.objects.get(name='3s')

In [2]:
manually_labeled = []
for labeler in tqdm(face_identity_labelers):
    name = labeler.name.split(':')[-1]
    manually_labeled.append({
        'name': name,
        'screentime': FaceIdentity.objects.filter(
            labeler=labeler, probability__gte=0.5, face__frame__sampler=sampler_3s
        ).count() * 3
    })
manually_labeled.sort(key=lambda x: x['screentime'])
print(len(manually_labeled))

100%|██████████████████████████████████████████████████████████| 264/264 [7:18:23<00:00,  7.97s/it]

264


In [3]:
rekognition_and_manually_labeled = []
for person in tqdm(manually_labeled):
    try:
        identity = Identity.objects.get(name=person['name'])
        rekognition_and_manually_labeled.append({
            'name': person['name'],
            'our_result': person['screentime'],
            'aws_result': FaceIdentity.objects.filter(
                identity=identity, labeler__in=rekognition_labelers,
                face__frame__sampler=sampler_3s,
                probability__gte=0.5
            ).count() * 3
        })
    except Exception as e:
        print(person, e)

100%|█████████████████████████████████████████████████████████| 264/264 [4:53:31<00:00, 936.55s/it]


In [4]:
print('name,our_result,aws_result')
for person in rekognition_and_manually_labeled:
    print(','.join([person['name'], str(person['our_result']), str(person['aws_result'])]))

name,our_result,aws_result
miley cyrus,18,6285
carl sagan,141,93
eric garner,423,0
chris harper-mercer,1509,0
neil armstrong,2949,4410
sonny perdue,3705,6501
keira knightley,4848,1770
walter scott,4878,45
willie corey godbolt,5217,0
bill cosby,7578,138555
devin patrick kelley,7701,0
samuel dubose,8616,0
roger ailes,10761,31746
esteban santiago-ruiz,11631,0
robert lewis dear jr,12093,0
micah xavier johnson,13383,0
neil degrasse tyson,17724,19080
kanye west,18084,21333
freddie gray,22386,0
alex jones,23010,33933
al franken,24813,157257
ryan zinke,29496,35772
george zimmerman,30249,465306
betsy devos,30795,45210
stephen paddock,31593,0
dorothy rabinowitz,35145,0
lincoln chafee,35520,46146
gavin eugene long,37227,0
omar mateen,41652,33510
justin trudeau,42513,28161
dylann roof,42966,0
gina haspel,43221,0
jim webb,46203,52098
lauren green,47502,43722
jim gilmore,47679,32319
syed rizwan farook,48684,0
tashfeen malik,48828,0
doug jones,54138,246
christian taylor,61101,90
muhammad youssef abdu

In [ ]:
# Slower check

rekognition_and_manually_labeled_detailed = []
for labeler in tqdm(face_identity_labelers):
    name = labeler.name.split(':')[-1]
    identity = Identity.objects.get(name=name)
    our_ids = {
        x['face__id'] for x in FaceIdentity.objects.filter(
            labeler=labeler, probability__gt=0.5,
            face__frame__sampler=sampler_3s
        ).values('face__id')
    }
    aws_ids = {
        x['face__id'] for x in FaceIdentity.objects.filter(
            identity=identity, labeler__in=rekognition_labelers,
            face__frame__sampler=sampler_3s,
            probability__gt=0.5
        ).values('face__id')
    }
    rekognition_and_manually_labeled_detailed.append({
        'name': name,
        'our_count': len(our_ids),
        'aws_count': len(aws_ids),
        'both_count': len(our_ids & aws_ids)
    })

In [ ]:
print('name,our_count,aws_count,both_count')
for person in rekognition_and_manually_labeled_detailed:
    print(','.join([person['name'], str(person['our_count']), str(person['aws_count']), str(person['both_count'])]))